In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
from utils import chart_price, plot, v_backtester, c_backtester, perf, perf_var, breakout_strategy, bootstrap, m_proc
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
import matplotlib.pyplot as plt
from typing import NamedTuple
from collections import namedtuple
%matplotlib inline

In [2]:
from datastore_pytables import Store

In [3]:
store = Store()
store.keys()

['/cont/min/CL_20191120_NYMEX_USD',
 '/cont/min/ES_20191220_GLOBEX_USD',
 '/cont/min/GC_20191227_NYMEX_USD',
 '/cont/min/GE_20191216_GLOBEX_USD',
 '/cont/min/NKD_20191212_GLOBEX_USD',
 '/cont/min/NQ_20191220_GLOBEX_USD',
 '/cont/min/YM_20191220_ECBOT_USD',
 '/cont/min/ZB_20191219_ECBOT_USD',
 '/cont/min/ZF_20191231_ECBOT_USD',
 '/cont/min/ZN_20191219_ECBOT_USD']

In [4]:
contract = store.read('/cont/min/CL_20191120_NYMEX_USD').sort_index()

In [5]:
table = bootstrap(contract,
          start = '20190101', 
          end = '20191231',
          paths=100)

In [6]:
func = partial(breakout_strategy, 
                  time_int = 30,
                  periods = [5, 10, 20, 40, 80, 160],
                  ema_fast = 5,
                  ema_slow = 120,
                  atr_periods = 180,
                  sl_atr = 1,
                  take_profit=0
                  )

In [7]:
results = m_proc(table, func)

In [8]:
res = pd.DataFrame([perf(i, output=False).stats for i in results])

In [9]:
res[res['Annual return'] > 0]['Annual return'].count()

91

In [10]:
res.sort_values('Annual return')

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,...,Avg gain/loss ratio,Position EV,Positions per day,Days per position,Actual avg. duration,Days,Positions,Trades,Monthly EV,Annual EV
21,-0.317578,-0.199802,0.306537,-1.092987,-1.079470,0.362164,-0.294198,0.827929,-1.481087,-0.031684,...,1.591834,-0.025292,2.564626,0.389920,03:54:00,147,377,754,-1.362141,-16.345694
31,-0.229452,-0.141052,0.303614,-0.706583,-0.639649,0.164936,-0.358715,0.877925,-0.975035,-0.190948,...,1.620932,-0.017876,2.523810,0.396226,04:00:00,147,371,742,-0.947443,-11.369313
58,-0.179835,-0.109209,0.303592,-0.500720,-0.884026,0.254100,-0.203427,0.915114,-0.684701,-0.519646,...,1.678399,-0.014358,2.387755,0.418803,03:47:00,147,351,702,-0.719964,-8.639572
78,-0.170346,-0.103212,0.280697,-0.527344,-0.748138,0.228284,-0.227694,0.912375,-0.813850,1.176899,...,1.504481,-0.013290,2.421769,0.412921,03:46:00,147,356,711,-0.675911,-8.110936
7,-0.117234,-0.070157,0.207924,-0.496417,-0.509715,0.229945,-0.230000,0.914634,-0.718575,0.214992,...,1.522058,-0.009110,2.380952,0.420000,03:40:00,147,350,699,-0.455499,-5.465983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1.257305,0.602721,0.238530,3.533578,26.612976,0.905981,-0.047244,2.103389,9.249044,3.009381,...,1.947496,0.089505,2.191781,0.456250,04:02:00,146,320,640,4.119693,49.436320
77,1.352268,0.647027,0.248137,3.575477,17.622691,0.927027,-0.076734,1.912309,6.699655,0.341024,...,1.960276,0.086274,2.244898,0.445455,04:13:00,147,330,660,4.067215,48.806581
25,1.368132,0.653497,0.187890,4.688954,33.372716,0.944685,-0.040996,2.195664,9.593400,0.325865,...,1.975995,0.097026,2.115646,0.472669,04:02:00,147,311,622,4.310731,51.728772
2,1.550542,0.726642,0.298520,3.288977,16.408520,0.898129,-0.094496,1.761483,6.304298,0.602524,...,2.258751,0.093029,2.340136,0.427326,04:02:00,147,344,687,4.571729,54.860744


In [11]:
res['Sharpe ratio'].median()

1.9756867672699063

In [12]:
res['Sharpe ratio'].mean()

2.0229085340197583

In [13]:
res[res['Annual return'] >=.2]['Annual return'].count()

83

In [14]:
res['Position EV'].median()

0.034531661598850835

In [15]:
res['Position EV'].mean()

0.037282083700558036

In [16]:
(res[res['Position EV'] < 2])['Annual return'].count()

100

In [17]:
(res[res['Position EV'] < 5])['Annual return'].count()

100

In [18]:
res['Monthly EV'].median()

1.6830609426359857

In [19]:
res['Monthly EV'].mean()

1.7843978310325987